14. Read Rewriting the particle simulator in Numpy on Chapter 2: Fast Array Operations with Numpy and Pandas (pp. 68) from the book G. Lenaro (2017). Python high Performance. Second Edition. UK: Packt Publishing Ltd. Implement the improvements on the particle simulator using NumPy. Show that both implementations scale linearly with particle size, but the runtime in the pure Python version grows much faster than the NumPy version. 

In [2]:
import numpy as np
from random import uniform
from timeit import timeit

class Particle:
    def __init__(self, x, y, velocidad_angular):
        self.x = x
        self.y = y
        self.velocidad_angular = velocidad_angular

class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.velocidad_angular * v_x
                d_y = timestep * p.velocidad_angular * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * velocidad_angular_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

 
def benchmark(npart=1000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

In [6]:
%timeit benchmark(10000, 'python')

38.4 s ± 817 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit benchmark(10000, 'numpy')

2.11 s ± 56.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


15. Explain how to optain the optimal performance with numexpr. Read the section Reaching optimal performance with numexpr, pp. 72 from the previous reference.
Implement it and measure the execution time.

In [8]:
import numexpr 

A = np.array([[0.0,0.0],[3.0,4.0]])
n = 1000
p = 3
xs = np.random.random((n, p))

def pdist_numpy(xs):
    return np.sqrt(((xs[:,None,:] - xs)**2).sum(-1))

print(pdist_numpy(xs))
%timeit pdist_numpy(xs)

[[0.         0.36515394 0.96830487 ... 0.73152256 0.45752217 0.96973047]
 [0.36515394 0.         0.63638963 ... 0.61872088 0.25351813 0.65449158]
 [0.96830487 0.63638963 0.         ... 0.63888009 0.70609669 0.34589871]
 ...
 [0.73152256 0.61872088 0.63888009 ... 0.         0.61414093 0.58475264]
 [0.45752217 0.25351813 0.70609669 ... 0.61414093 0.         0.59025566]
 [0.96973047 0.65449158 0.34589871 ... 0.58475264 0.59025566 0.        ]]
30.1 ms ± 3.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
def pdist_numexpr(xs):
    a = xs[:, np.newaxis, :]
    return np.sqrt(numexpr.evaluate('sum((a-xs)**2, axis=2)'))

print(pdist_numexpr(xs))
%timeit pdist_numexpr(xs)

[[0.         0.36515394 0.96830487 ... 0.73152256 0.45752217 0.96973047]
 [0.36515394 0.         0.63638963 ... 0.61872088 0.25351813 0.65449158]
 [0.96830487 0.63638963 0.         ... 0.63888009 0.70609669 0.34589871]
 ...
 [0.73152256 0.61872088 0.63888009 ... 0.         0.61414093 0.58475264]
 [0.45752217 0.25351813 0.70609669 ... 0.61414093 0.         0.59025566]
 [0.96973047 0.65449158 0.34589871 ... 0.58475264 0.59025566 0.        ]]
10.1 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
